In [8]:
from graphviz import Digraph
import time

# Grafo
def acao(destino, custo):
    return {'destino': destino, 'custo': custo}

estados_romenia = [
    { 'estado': 'Arad', 'acoes': [acao('Zerind', 75), acao('Sibiu', 140), acao('Timisoara', 118)] },
    { 'estado': 'Zerind', 'acoes': [acao('Arad', 75), acao('Oradea', 71)] },
    { 'estado': 'Timisoara', 'acoes': [acao('Arad', 118), acao('Lugoj', 111)] },
    { 'estado': 'Sibiu', 'acoes': [acao('Arad', 140), acao('Oradea', 151), acao('Fagaras', 99), acao('Rimnicu Vilcea', 80)] },
    { 'estado': 'Oradea', 'acoes': [acao('Zerind', 71), acao('Sibiu', 151)] },
    { 'estado': 'Lugoj', 'acoes': [acao('Timisoara', 111), acao('Mehadia', 70)] },
    { 'estado': 'Mehadia', 'acoes': [acao('Lugoj', 70), acao('Drobeta', 75)] },
    { 'estado': 'Drobeta', 'acoes': [acao('Mehadia', 75), acao('Craiova', 120)] },
    { 'estado': 'Craiova', 'acoes': [acao('Drobeta', 120), acao('Rimnicu Vilcea', 146), acao('Pitesti', 138)] },
    { 'estado': 'Rimnicu Vilcea', 'acoes': [acao('Sibiu', 80), acao('Craiova', 146), acao('Pitesti', 97)] },
    { 'estado': 'Fagaras', 'acoes': [acao('Sibiu', 99), acao('Bucharest', 211)] },
    { 'estado': 'Pitesti', 'acoes': [acao('Rimnicu Vilcea', 97), acao('Craiova', 138), acao('Bucharest', 101)] },
    { 'estado': 'Giurgiu', 'acoes': [acao('Bucharest', 90)] },
    { 'estado': 'Bucharest', 'acoes': [acao('Fagaras', 211), acao('Pitesti', 101), acao('Giurgiu', 90), acao('Urziceni', 85)] },
    { 'estado': 'Urziceni', 'acoes': [acao('Bucharest', 85), acao('Vaslui', 142), acao('Hirsova', 98)] },
    { 'estado': 'Hirsova', 'acoes': [acao('Urziceni', 98), acao('Eforie', 86)] },
    { 'estado': 'Eforie', 'acoes': [acao('Hirsova', 86)] },
    { 'estado': 'Vaslui', 'acoes': [acao('Urziceni', 142), acao('Iasi', 92)] },
    { 'estado': 'Iasi', 'acoes': [acao('Vaslui', 92), acao('Neamt', 87)] },
    { 'estado': 'Neamt', 'acoes': [acao('Iasi', 87)] }
]

# Material do código fonte
class No:
    def __init__(self, estado, custo, pai, acao):
        self.estado = estado
        self.custo = custo
        self.pai = pai
        self.acao = acao
        self.id = id(self)

    def filhos(self, problema):
        espaco_acoes = next(e for e in problema.espaco_estados if e['estado'] == self.estado)
        return [No(acao['destino'], self.custo + acao['custo'], self, acao['destino']) for acao in espaco_acoes['acoes']]

    def constroi_solucao(self):
        no_atual = self
        solucao = [no_atual]
        while no_atual.pai:
            no_atual = no_atual.pai
            solucao.insert(0, no_atual)
        return solucao

class Problema:
    def __init__(self, espaco_estados, inicial, objetivo):
        self.espaco_estados = espaco_estados
        self.inicial = inicial
        self.objetivo = objetivo

In [9]:
BUSCA_INICIANDO = 0
BUSCA_FALHOU = 1
BUSCA_SUCESSO = 2

# Largura
class BuscaLargura:
    def __init__(self, problema):
        self.problema = problema
        self.fronteira = [problema.inicial]
        self.visitados = [problema.inicial.estado]
        self.solucao = []
        self.situacao = BUSCA_INICIANDO

    def mostrar_passo(self, no):
        print("\n📍 Expandindo:", no.estado)
        print("   Fronteira:", [n.estado for n in self.fronteira])
        print("   Visitados:", self.visitados)

    def passo_busca(self):
        if self.situacao in [BUSCA_SUCESSO, BUSCA_FALHOU]:
            return
        if not self.fronteira:
            self.situacao = BUSCA_FALHOU
            return
        no = self.fronteira.pop(0)
        self.mostrar_passo(no)
        if self.problema.objetivo(no):
            self.situacao = BUSCA_SUCESSO
            self.solucao = no.constroi_solucao()
            return
        for filho in no.filhos(self.problema):
            if filho.estado not in self.visitados:
                self.fronteira.append(filho)
                self.visitados.append(filho.estado)

    def executar(self):
        while self.situacao not in [BUSCA_SUCESSO, BUSCA_FALHOU]:
            self.passo_busca()

# Profundidade
class BuscaProfundidade:
    def __init__(self, problema):
        self.problema = problema
        self.fronteira = [problema.inicial]
        self.visitados = [problema.inicial.estado]
        self.solucao = []
        self.situacao = BUSCA_INICIANDO

    def mostrar_passo(self, no):
        print("\nNó-> Expandindo:", no.estado)
        print("   Fronteira:", [n.estado for n in self.fronteira])
        print("   Visitados:", self.visitados)

    def passo_busca(self):
        if self.situacao in [BUSCA_SUCESSO, BUSCA_FALHOU]:
            return
        if not self.fronteira:
            self.situacao = BUSCA_FALHOU
            return
        no = self.fronteira.pop()
        self.mostrar_passo(no)
        if self.problema.objetivo(no):
            self.situacao = BUSCA_SUCESSO
            self.solucao = no.constroi_solucao()
            return
        for filho in reversed(no.filhos(self.problema)):
            if filho.estado not in self.visitados:
                self.fronteira.append(filho)
                self.visitados.append(filho.estado)

    def executar(self):
        while self.situacao not in [BUSCA_SUCESSO, BUSCA_FALHOU]:
            self.passo_busca()

# Desenho do caminho
def desenhar_arvore(solucao):
    dot = Digraph(comment='Árvore de Busca')
    for no in solucao:
        label = f"{no.estado}\n(custo: {no.custo})"
        dot.node(str(no.id), label)
        if no.pai:
            dot.edge(str(no.pai.id), str(no.id))
    return dot

In [10]:
# Teste
tipo_busca = "largura"  # largura ou "profundidade"
origem = "Arad"
destino = "Bucharest"

no_inicial = No(origem, 0, None, None)
problema = Problema(estados_romenia, no_inicial, lambda no: no.estado == destino)

print(f"🔎 Buscando de {origem} para {destino} usando {tipo_busca.upper()}...\n")

if tipo_busca == "largura":
    busca = BuscaLargura(problema)
elif tipo_busca == "profundidade":
    busca = BuscaProfundidade(problema)
else:
    raise ValueError("Tipo de busca inválido.")

inicio_tempo = time.time()
busca.executar()
fim_tempo = time.time()
tempo_execucao = fim_tempo - inicio_tempo


print(f"\n Nós visitados: {len(busca.visitados)}")
print(f" Profundidade da solução: {len(busca.solucao)}")
print(f" Custo total da solução: {busca.solucao[-1].custo}")
print(f" Tempo de execução: {tempo_execucao:.4f} segundos")

if busca.situacao == BUSCA_SUCESSO:
    print("\n✅ Caminho encontrado:")
    for no in busca.solucao:
        print(f"→ {no.estado} (custo: {no.custo})")
        print("\n\n")

    dot = desenhar_arvore(busca.solucao)
    dot.render('busca_romenia', format='png', cleanup=True)
    from IPython.display import Image
    display(Image(filename='busca_romenia.png'))
else:
    print("Busca falhou.")

🔎 Buscando de Arad para Bucharest usando LARGURA...


📍 Expandindo: Arad
   Fronteira: []
   Visitados: ['Arad']

📍 Expandindo: Zerind
   Fronteira: ['Sibiu', 'Timisoara']
   Visitados: ['Arad', 'Zerind', 'Sibiu', 'Timisoara']

📍 Expandindo: Sibiu
   Fronteira: ['Timisoara', 'Oradea']
   Visitados: ['Arad', 'Zerind', 'Sibiu', 'Timisoara', 'Oradea']

📍 Expandindo: Timisoara
   Fronteira: ['Oradea', 'Fagaras', 'Rimnicu Vilcea']
   Visitados: ['Arad', 'Zerind', 'Sibiu', 'Timisoara', 'Oradea', 'Fagaras', 'Rimnicu Vilcea']

📍 Expandindo: Oradea
   Fronteira: ['Fagaras', 'Rimnicu Vilcea', 'Lugoj']
   Visitados: ['Arad', 'Zerind', 'Sibiu', 'Timisoara', 'Oradea', 'Fagaras', 'Rimnicu Vilcea', 'Lugoj']

📍 Expandindo: Fagaras
   Fronteira: ['Rimnicu Vilcea', 'Lugoj']
   Visitados: ['Arad', 'Zerind', 'Sibiu', 'Timisoara', 'Oradea', 'Fagaras', 'Rimnicu Vilcea', 'Lugoj']

📍 Expandindo: Rimnicu Vilcea
   Fronteira: ['Lugoj', 'Bucharest']
   Visitados: ['Arad', 'Zerind', 'Sibiu', 'Timisoara', 'Oradea'

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [ ]:
from copy import deepcopy
from collections import deque

# print  do sudoku
def print_sudoku(board):
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-" * 21)
        row = ""
        for j in range(9):
            if j % 3 == 0 and j != 0:
                row += "| "
            row += str(board[i][j]) + " "
        print(row)
    print()

# Verificação da condição do sudoku
def valido(board, num, pos):
    linha, coluna = pos
    if num in board[linha]: return False
    if num in [board[i][coluna] for i in range(9)]: return False
    bloco_x, bloco_y = (coluna // 3) * 3, (linha // 3) * 3
    for i in range(bloco_y, bloco_y + 3):
        for j in range(bloco_x, bloco_x + 3):
            if board[i][j] == num:
                return False
    return True

def encontrar_vazio(board):
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return (i, j)
    return None

# Sudoku
class NoSudoku:
    def __init__(self, board, profundidade=0):
        self.board = board
        self.profundidade = profundidade

    def estado_tuple(self):
        return tuple(tuple(row) for row in self.board)

In [ ]:
# DFS
class BuscaSudokuDFS:
    def __init__(self, board):
        self.board = board
        self.visitados = 0
        self.solucao = None
        self.tempo = 0

    def resolver(self):
        inicio = time.time()
        sucesso = self._dfs(self.board)
        fim = time.time()
        self.tempo = fim - inicio
        return sucesso

    def _dfs(self, board):
        vazio = encontrar_vazio(board)
        if not vazio:
            self.solucao = board
            return True

        linha, coluna = vazio
        for num in range(1, 10):
            if valido(board, num, (linha, coluna)):
                board[linha][coluna] = num
                self.visitados += 1

                if self._dfs(board):
                    return True

                board[linha][coluna] = 0  # backtrack
        return False

# BFS
class BuscaSudokuBFS:
    def __init__(self, board):
        self.board = board
        self.visitados = 0
        self.solucao = None
        self.tempo = 0

    def resolver(self):
        inicio = time.time()
        fila = deque()
        fila.append(NoSudoku(deepcopy(self.board)))

        while fila:
            no = fila.popleft()
            self.visitados += 1

            vazio = encontrar_vazio(no.board)
            if not vazio:
                self.solucao = no.board
                self.tempo = time.time() - inicio
                return True

            linha, coluna = vazio
            for num in range(1, 10):
                if valido(no.board, num, (linha, coluna)):
                    novo_tabuleiro = deepcopy(no.board)
                    novo_tabuleiro[linha][coluna] = num
                    filho = NoSudoku(novo_tabuleiro, no.profundidade + 1)
                    fila.append(filho)

        self.tempo = time.time() - inicio
        return False

sudoku_teste = [
    [0, 0, 0, 2, 6, 0, 7, 0, 1],
    [6, 8, 0, 0, 7, 0, 0, 9, 0],
    [1, 9, 0, 0, 0, 4, 5, 0, 0],
    [8, 2, 0, 1, 0, 0, 0, 4, 0],
    [0, 0, 4, 6, 0, 2, 9, 0, 0],
    [0, 5, 0, 0, 0, 3, 0, 2, 8],
    [0, 0, 9, 3, 0, 0, 0, 7, 4],
    [0, 4, 0, 0, 5, 0, 0, 3, 6],
    [7, 0, 3, 0, 1, 8, 0, 0, 0]
]

In [ ]:
# Comparação

# DFS
dfs = BuscaSudokuDFS(deepcopy(sudoku_teste))
dfs.resolver()

# BFS
bfs = BuscaSudokuBFS(deepcopy(sudoku_teste))
bfs.resolver()

# Resultador
print("\n🧪 COMPARAÇÃO ENTRE BUSCA EM PROFUNDIDADE E LARGURA")
print("-" * 50)
print(" PROFUNDIDADE (DFS)")
print(f" Tempo: {dfs.tempo:.4f} s")
print(f" Nós visitados: {dfs.visitados}")
print(" Solução encontrada:" if dfs.solucao else "Falhou")
if dfs.solucao:
    print_sudoku(dfs.solucao)

print("-" * 50)
print(" LARGURA (BFS)")
print(f" Tempo: {bfs.tempo:.4f} s")
print(f" Nós visitados: {bfs.visitados}")
print(" Solução encontrada:" if bfs.solucao else "Falhou")
if bfs.solucao:
    print_sudoku(bfs.solucao)